In [1]:
import numpy as np
import cv2
from numpy import load
from numpy import zeros
from numpy import ones
from numpy.random import randint
from numpy.random import randn
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import RandomNormal
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import RepeatVector
from tensorflow.keras.layers import Embedding
from matplotlib import pyplot
from tensorflow.keras.utils import plot_model

import tensorflow as tf

In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  # Restrict TensorFlow to only allocate 11GB of memory on the first GPU
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=10240)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Virtual devices must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [3]:
# define the discriminator model
def define_discriminator(image_shape, vector_shape, n_classes=7):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # source image input
    in_src_image = Input(shape=image_shape)
    # vector input
    in_target = Input(shape=(vector_shape[0], vector_shape[1]))
    # label input
    in_label = Input(shape=(1,))
    # Vector layer
    n_nodes = (image_shape[0] * image_shape[1] * image_shape[2])
    # embedding for categorical input
    li = Embedding(n_classes, 50)(in_label)
    # scale up to image dimensions with linear
    li = Dense(n_nodes)(li)
    # reshape to additional channel
    li = Reshape((image_shape[0], image_shape[1], image_shape[2]))(li)
    # LSTM Model
    p = LSTM(200, activation='relu')(in_target)
    p = Dense(n_nodes)(p)
    # Reshape and mergo to image dimension
    p = Reshape((image_shape[0], image_shape[1], image_shape[2]))(p)
    merged = Concatenate()([p, in_src_image, li])
    # concatenate images channel-wise
    # C64
    d = Conv2D(64, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(merged)
    d = LeakyReLU(alpha=0.2)(d)
    # C128
    d = Conv2D(128, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # C256
    d = Conv2D(256, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # C512
    d = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # second last output layer
    d = Conv2D(512, (4,4), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    # patch output
    d = Conv2D(1, (4,4), padding='same', kernel_initializer=init)(d)
    d = BatchNormalization()(d)
    d = LeakyReLU(alpha=0.2)(d)
    d = Flatten()(d)
    d = Dropout(0.4)(d)
    out_layer = Dense(1, activation='sigmoid')(d)
    # define model
    model = Model([in_src_image, in_target, in_label], out_layer)
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=opt, loss_weights=[0.5])
    return model

In [4]:
# define an encoder block
def define_encoder_block(layer_in, n_filters, batchnorm=True):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # add downsampling layer
    g = Conv2D(n_filters, (4,4), strides=(2,2), padding='same',
    kernel_initializer=init)(layer_in)
    # conditionally add batch normalization
    if batchnorm:
        g = BatchNormalization()(g, training=True)
    # leaky relu activation
    g = LeakyReLU(alpha=0.2)(g)
    return g

In [5]:
# define the standalone generator model
def define_generator(in_shape, vector_shape, latent_dim, n_classes=7):
    # weight initialization
    init = RandomNormal(stddev=0.02)
    # image input
    in_image = Input(shape=in_shape)
    in_lat = Input(shape=latent_dim)
    in_label = Input(shape=(1,))
    # embedding for categorical input
    li = Embedding(n_classes, 50)(in_label)
    li = Dense(latent_dim)(li)
    li = Reshape([latent_dim])(li)
    gen = LeakyReLU(alpha=0.2)(in_lat)
    # merge image gen and label input
    # encoder model
    e1 = define_encoder_block(in_image, 64, batchnorm=False)
    e2 = define_encoder_block(e1, 128)
    e3 = define_encoder_block(e2, 256)
    e4 = define_encoder_block(e3, 512)
    e5 = define_encoder_block(e4, 512)
    e6 = define_encoder_block(e5, 512)
    e7 = define_encoder_block(e6, 512)
    # bottleneck, no batch norm and relu
    b = Conv2D(512, (4,4), strides=(2,2), padding='same', kernel_initializer=init)(e7)
    b = Activation('relu')(b)
    b = Flatten()(b)
    # Merge latent space variable input with network
    b = Concatenate()([b, gen, li])
    # Reshape to vector size
    n_nodes = (vector_shape[0] * vector_shape[1])
    b = Dense(n_nodes)(b)
    b = Reshape([vector_shape[0], vector_shape[1]])(b)
    # encoder-decoder LSTM model
    d = LSTM(200, activation='relu')(b)
    d = RepeatVector(vector_shape[0])(d)
    d = LSTM(200, activation='relu', return_sequences=True)(d)
    # output
    out_layer = Dense(3)(d)

    # define model input & output
    model = Model([in_image, in_lat, in_label], out_layer)
    return model

In [6]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model, image_shape, vector_shape, latent_dim):
    # make weights in the discriminator not trainable
    for layer in d_model.layers:
        if not isinstance(layer, BatchNormalization):
            layer.trainable = False
    # define the source image
    in_src = Input(shape=image_shape)
    in_lat = Input(shape=latent_dim)
    in_label = Input(shape=(1,))
    # connect the source image to the generator input
    gen_out = g_model([in_src, in_lat, in_label])
    # connect the source input and generator output to the discriminator input
    dis_out = d_model([in_src, gen_out, in_label])
    # src image as input, generated image and classification output
    model = Model([in_src, in_lat, in_label], [dis_out, gen_out])
    # compile model
    opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss=['binary_crossentropy', 'mae'], optimizer=opt, loss_weights=[1,100])
    return model

In [7]:
# load and prepare training images
def load_real_samples(filename):
	# load compressed arrays
	data = load(filename)
	# unpack arrays
	X1, X2, X3 = data['arr_0'], data['arr_1'], data['arr_2']
	return [X1, X2, X3-1]

In [8]:
# select a batch of random samples, returns images and target
def generate_real_samples(dataset, n_samples):
	# unpack dataset
	img, vec, label = dataset
	# choose random instances
	ix = randint(0, img.shape[0], n_samples)
	# retrieve selected images
	X1, X2, X3 = img[ix], vec[ix], label[ix]
	# generate 'real' class labels 
	y = ones((n_samples, 1))
	return [X1, X2, X3], y

In [9]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape(n_samples, latent_dim)
    return x_input


In [10]:
# generate a batch of images, returns images and targets
def generate_fake_samples(g_model, samples, label, n_samples):
    # Generate latent space points
    z_input = generate_latent_points(latent_dim, n_samples)
    # generate fake instance
    X = g_model.predict([samples, z_input, label])
    # create 'fake' class labels 
    y = zeros((n_samples, 1))
    return X, y

In [11]:
# Overlay trajectories (data_y) to the image (data_x)
def create_trajectory(data_x, data_y, obs_len=10):
    # Calibration parameter to overlay for a 1280x360 resolution image
    K = np.array([[537.023764, 0, 640 , 0], 
                    [0 , 537.023764, 180, 0], 
                    [0, 0, 1, 0]])
    # Rotation matrix to obtain egocentric trajectory
    Rt = np.array([[0.028841, 0.007189, 0.999558, 1.481009],
                    [-0.999575,  0.004514,  0.028809,  0.296583],
                    [ 0.004305,  0.999964, -0.007316, -1.544537],
                    [ 0.      ,  0.      ,  0.      ,  1.      ]])

    # Resize data back to 1280x360
    data_x = cv2.resize(data_x, (1280,360))
    # Add column of ones for rotation matrix multiplication
    data_y = np.hstack((data_y, np.ones((len(data_y),1))))
    # Draw points
    for m in range(obs_len, data_y.shape[0]):
        # Rotation matrix multiplication of trajectory 
        A = np.matmul(np.linalg.inv(Rt), data_y[m, :].reshape(4, 1))
        # Egocentric view of trajectory
        B = np.matmul(K, A)
        # Circle location of trajectories 
        x = int(B[0, 0] * 1.0 / B[2, 0])
        y = int(B[1, 0] * 1.0 / B[2, 0])
        if (x < 0 or x > 1280 - 1 or y > 360 - 1):
            continue
        # Use opencv to overlay trajectories
        data_x = cv2.circle(data_x, (x, y), 3, (0, 0, 255), -1)
    return data_x

In [12]:
def generate_drvact_text(drvact):
    text = '[warnings] drvact label is not defined ...'
    if (drvact == 1):
        text = 'Go'
    elif (drvact == 2):
        text = 'Turn Left'
    elif (drvact == 3):
        text = 'Turn Right'
    elif (drvact == 4):
        text = 'U-turn'
    elif (drvact == 5):
        text = 'Left LC'
    elif (drvact == 6):
        text = 'Right LC'
    elif (drvact == 7):
        text = 'Avoidance'
    return text


In [13]:
# generate samples and save as a plot and save the model
def summarize_performance(step, g_model, dataset, n_samples=1):
    # select a sample of input images
    [X_real_img, X_real_vec, X_label], _ = generate_real_samples(dataset, n_samples)
    # generate a batch of fake samples
    X_fake_vec, _ = generate_fake_samples(g_model, X_real_img, X_label, n_samples)
    # scale all pixels from [-1,1] to [0,1]
    X_real_img = (X_real_img + 1) / 2.0
    pyplot.figure(figsize=(32.0, 20.0))
    # plot real source images
    for i in range(n_samples):
        orig_image = (X_real_img[i]* 255).astype(np.uint8)
        orig_image = cv2.resize(orig_image, (1280,360))
        pyplot.subplot(3, n_samples, 1 + i)
        pyplot.axis('off')
        pyplot.imshow(orig_image)
    # plot generated target image
    for i in range(n_samples):
        fake_sample = create_trajectory((X_real_img[i]* 255).astype(np.uint8), X_fake_vec[i])
        pyplot.subplot(3, n_samples, 1 + n_samples + i)
        pyplot.axis('off')
        pyplot.imshow(fake_sample)
    # plot real target image
    for i in range(n_samples):
        true_sample = create_trajectory((X_real_img[i]* 255).astype(np.uint8), X_real_vec[i])
        pyplot.subplot(3, n_samples, 1 + n_samples*2 + i)
        pyplot.axis('off')
        pyplot.imshow(true_sample)
    # save plot to file
    filename1 = 'plot_%06d.png' % (step+1)
    pyplot.savefig(filename1)
    pyplot.close()
    # save the generator model
    filename2 = 'model_%06d.h5' % (step+1)
    g_model.save(filename2)
    print('>Saved: %s and %s' % (filename1, filename2))

In [14]:
# train models
def train(d_model, g_model, gan_model, dataset, latent_dim, n_epochs=10, n_batch=3):
    # calculate the number of batches per training epoch
    bat_per_epo = int(len(dataset[0]) / n_batch)
    # calculate the number of training iterations
    n_steps = bat_per_epo * n_epochs
    # manually enumerate epochs
    for i in range(n_steps):
        # select a batch of real samples
        [X_real_img, X_real_vec, X_label], y_real = generate_real_samples(dataset, n_batch)
        # generate a batch of fake samples
        X_fake_vec, y_fake = generate_fake_samples(g_model, X_real_img, X_label, n_batch)
        # update discriminator for real samples
        d_loss1 = d_model.train_on_batch([X_real_img, X_real_vec, X_label], y_real)
        # update discriminator for generated samples
        d_loss2 = d_model.train_on_batch([X_real_img, X_fake_vec, X_label], y_fake)
        X_lat = generate_latent_points(latent_dim, n_batch)
        # update the generator
        g_loss, _, _ = gan_model.train_on_batch([X_real_img, X_lat, X_label], [y_real, X_real_vec])
        # summarize performance
        print('>%d, d1[%.3f] d2[%.3f] g[%.3f]' % (i+1, d_loss1, d_loss2, g_loss))
        # summarize model performance
        if (i+1) % int(bat_per_epo) == 0:
            summarize_performance(i, g_model, dataset)
            print('saved')

In [15]:
%%time
# load image data
dataset = load_real_samples('./dual_condition_dataset_train.npz')
print('Loaded', dataset[0].shape, dataset[1].shape, dataset[2].shape)

Loaded (16805, 256, 256, 3) (16805, 40, 3) (16805,)
CPU times: user 22.9 s, sys: 1.39 s, total: 24.3 s
Wall time: 24.3 s


In [16]:
# define input shape based on the loaded dataset
image_shape = dataset[0].shape[1:]
vector_shape = dataset[1].shape[1:]
latent_dim = 512
# %%
# define the models
d_model = define_discriminator(image_shape, vector_shape)
g_model = define_generator(image_shape, vector_shape, latent_dim)
# %%
# define the composite model
gan_model = define_gan(g_model, d_model, image_shape, vector_shape, latent_dim)

In [17]:
# train model
train(d_model, g_model, gan_model, dataset, latent_dim)

>1, d1[0.499] d2[1.823] g[552.942]
>2, d1[0.152] d2[1.023] g[268.053]
>3, d1[0.288] d2[0.559] g[537.101]
>4, d1[0.160] d2[0.596] g[422.029]
>5, d1[0.182] d2[0.741] g[195.068]
>6, d1[0.081] d2[0.530] g[282.563]
>7, d1[0.124] d2[0.638] g[360.566]
>8, d1[0.131] d2[0.586] g[191.141]
>9, d1[0.088] d2[0.358] g[381.600]
>10, d1[0.051] d2[0.397] g[400.058]
>11, d1[0.192] d2[0.362] g[303.391]
>12, d1[0.029] d2[0.156] g[373.756]
>13, d1[0.004] d2[0.336] g[418.204]
>14, d1[0.043] d2[0.243] g[364.800]
>15, d1[0.053] d2[1.770] g[258.682]
>16, d1[0.295] d2[0.419] g[757.470]
>17, d1[0.478] d2[0.316] g[457.804]
>18, d1[0.524] d2[0.171] g[365.923]
>19, d1[0.393] d2[0.377] g[431.037]
>20, d1[0.307] d2[0.509] g[348.691]
>21, d1[0.333] d2[0.370] g[261.912]
>22, d1[0.357] d2[0.234] g[269.071]
>23, d1[0.097] d2[0.159] g[397.856]
>24, d1[0.441] d2[0.224] g[258.941]
>25, d1[0.236] d2[0.301] g[304.570]
>26, d1[0.388] d2[0.290] g[194.859]
>27, d1[0.205] d2[0.341] g[195.763]
>28, d1[0.081] d2[0.296] g[164.116]
>

>227, d1[0.627] d2[0.290] g[149.009]
>228, d1[0.358] d2[0.428] g[182.778]
>229, d1[0.366] d2[0.494] g[138.349]
>230, d1[0.237] d2[0.474] g[112.488]
>231, d1[0.269] d2[0.448] g[283.252]
>232, d1[0.284] d2[0.468] g[126.056]
>233, d1[0.290] d2[0.448] g[144.770]
>234, d1[0.331] d2[0.543] g[246.000]
>235, d1[0.300] d2[0.488] g[135.026]
>236, d1[0.329] d2[0.311] g[181.848]
>237, d1[0.294] d2[0.325] g[129.180]
>238, d1[0.379] d2[0.382] g[190.364]
>239, d1[0.495] d2[0.398] g[121.501]
>240, d1[0.379] d2[0.406] g[194.351]
>241, d1[0.436] d2[0.308] g[168.098]
>242, d1[0.322] d2[0.400] g[231.221]
>243, d1[0.329] d2[0.450] g[243.387]
>244, d1[0.398] d2[0.447] g[142.498]
>245, d1[0.326] d2[0.383] g[140.736]
>246, d1[0.403] d2[0.333] g[130.469]
>247, d1[0.367] d2[0.206] g[204.137]
>248, d1[0.365] d2[0.295] g[93.274]
>249, d1[0.377] d2[0.265] g[177.471]
>250, d1[0.287] d2[0.307] g[156.432]
>251, d1[0.371] d2[0.307] g[107.335]
>252, d1[0.390] d2[0.524] g[160.251]
>253, d1[0.465] d2[0.335] g[175.138]
>2

>451, d1[0.468] d2[0.265] g[80.616]
>452, d1[0.085] d2[0.162] g[150.134]
>453, d1[0.663] d2[0.181] g[107.556]
>454, d1[0.087] d2[0.142] g[112.628]
>455, d1[0.074] d2[0.098] g[283.614]
>456, d1[0.052] d2[0.112] g[119.119]
>457, d1[0.048] d2[0.041] g[190.763]
>458, d1[0.041] d2[0.058] g[223.280]
>459, d1[0.066] d2[0.082] g[124.489]
>460, d1[0.058] d2[0.062] g[145.162]
>461, d1[0.028] d2[0.145] g[105.600]
>462, d1[0.783] d2[0.063] g[175.300]
>463, d1[0.114] d2[0.052] g[136.237]
>464, d1[1.011] d2[0.154] g[47.834]
>465, d1[0.055] d2[0.122] g[190.475]
>466, d1[0.026] d2[0.182] g[207.885]
>467, d1[0.043] d2[0.099] g[177.491]
>468, d1[0.031] d2[0.106] g[158.638]
>469, d1[0.022] d2[0.107] g[126.497]
>470, d1[0.563] d2[1.480] g[121.310]
>471, d1[0.124] d2[0.670] g[287.509]
>472, d1[0.327] d2[0.579] g[160.242]
>473, d1[0.217] d2[0.474] g[190.674]
>474, d1[0.238] d2[0.463] g[129.307]
>475, d1[0.301] d2[0.416] g[145.610]
>476, d1[0.291] d2[0.450] g[161.325]
>477, d1[0.229] d2[0.411] g[194.158]
>47

>675, d1[0.332] d2[0.350] g[151.418]
>676, d1[0.362] d2[0.324] g[122.878]
>677, d1[0.324] d2[0.317] g[90.311]
>678, d1[0.344] d2[0.316] g[46.687]
>679, d1[0.341] d2[0.358] g[125.544]
>680, d1[0.371] d2[0.379] g[103.776]
>681, d1[0.331] d2[0.339] g[91.489]
>682, d1[0.329] d2[0.402] g[117.815]
>683, d1[0.306] d2[0.392] g[97.995]
>684, d1[0.302] d2[0.384] g[117.750]
>685, d1[0.290] d2[0.378] g[99.055]
>686, d1[0.351] d2[0.349] g[104.359]
>687, d1[0.323] d2[0.476] g[193.588]
>688, d1[0.293] d2[0.370] g[183.233]
>689, d1[0.332] d2[0.371] g[64.913]
>690, d1[0.303] d2[0.338] g[103.603]
>691, d1[0.332] d2[0.338] g[159.419]
>692, d1[0.325] d2[0.402] g[143.024]
>693, d1[0.285] d2[0.336] g[88.898]
>694, d1[0.368] d2[0.367] g[77.047]
>695, d1[0.341] d2[0.368] g[150.609]
>696, d1[0.291] d2[0.423] g[84.600]
>697, d1[0.316] d2[0.401] g[137.490]
>698, d1[0.291] d2[0.384] g[120.761]
>699, d1[0.296] d2[0.432] g[102.553]
>700, d1[0.363] d2[0.303] g[142.520]
>701, d1[0.354] d2[0.359] g[90.972]
>702, d1[0.

>899, d1[0.382] d2[0.357] g[76.383]
>900, d1[0.350] d2[0.342] g[163.092]
>901, d1[0.378] d2[0.336] g[160.991]
>902, d1[0.308] d2[0.310] g[118.051]
>903, d1[0.320] d2[0.329] g[122.061]
>904, d1[0.331] d2[0.364] g[161.493]
>905, d1[0.345] d2[0.350] g[107.040]
>906, d1[0.345] d2[0.348] g[145.950]
>907, d1[0.318] d2[0.317] g[164.587]
>908, d1[0.337] d2[0.339] g[76.516]
>909, d1[0.344] d2[0.334] g[87.426]
>910, d1[0.336] d2[0.347] g[94.509]
>911, d1[0.347] d2[0.388] g[117.265]
>912, d1[0.384] d2[0.333] g[154.809]
>913, d1[0.320] d2[0.375] g[131.260]
>914, d1[0.353] d2[0.346] g[81.458]
>915, d1[0.310] d2[0.344] g[139.932]
>916, d1[0.360] d2[0.333] g[133.341]
>917, d1[0.331] d2[0.321] g[119.979]
>918, d1[0.323] d2[0.322] g[87.283]
>919, d1[0.344] d2[0.315] g[127.159]
>920, d1[0.352] d2[0.357] g[125.637]
>921, d1[0.326] d2[0.382] g[84.746]
>922, d1[0.316] d2[0.325] g[72.585]
>923, d1[0.323] d2[0.341] g[109.437]
>924, d1[0.325] d2[0.359] g[105.695]
>925, d1[0.321] d2[0.355] g[144.745]
>926, d1[

...

>15825, d1[0.353] d2[0.352] g[22.059]
>15826, d1[0.339] d2[0.340] g[27.687]
>15827, d1[0.344] d2[0.364] g[27.921]
>15828, d1[0.342] d2[0.357] g[14.811]
>15829, d1[0.346] d2[0.359] g[31.423]
>15830, d1[0.353] d2[0.350] g[21.122]
>15831, d1[0.351] d2[0.350] g[25.495]
>15832, d1[0.349] d2[0.349] g[28.236]
>15833, d1[0.346] d2[0.340] g[29.200]
>15834, d1[0.345] d2[0.353] g[16.508]
>15835, d1[0.346] d2[0.349] g[19.003]
>15836, d1[0.317] d2[0.355] g[27.392]
>15837, d1[0.353] d2[0.356] g[23.737]
>15838, d1[0.355] d2[0.336] g[32.748]
>15839, d1[0.334] d2[0.359] g[35.996]
>15840, d1[0.331] d2[0.361] g[38.550]
>15841, d1[0.346] d2[0.343] g[32.730]
>15842, d1[0.336] d2[0.350] g[19.613]
>15843, d1[0.350] d2[0.355] g[52.369]
>15844, d1[0.351] d2[0.351] g[33.969]
>15845, d1[0.360] d2[0.349] g[21.343]
>15846, d1[0.342] d2[0.358] g[22.445]
>15847, d1[0.343] d2[0.390] g[34.871]
>15848, d1[0.337] d2[0.348] g[19.646]
>15849, d1[0.350] d2[0.341] g[20.947]
>15850, d1[0.338] d2[0.356] g[31.018]
>15851, d1[0

>16041, d1[0.348] d2[0.340] g[25.080]
>16042, d1[0.367] d2[0.343] g[15.750]
>16043, d1[0.351] d2[0.350] g[23.903]
>16044, d1[0.367] d2[0.335] g[23.030]
>16045, d1[0.358] d2[0.342] g[26.417]
>16046, d1[0.353] d2[0.338] g[26.326]
>16047, d1[0.354] d2[0.356] g[29.290]
>16048, d1[0.356] d2[0.335] g[28.698]
>16049, d1[0.340] d2[0.341] g[28.999]
>16050, d1[0.342] d2[0.339] g[21.391]
>16051, d1[0.342] d2[0.348] g[48.567]
>16052, d1[0.334] d2[0.344] g[23.536]
>16053, d1[0.334] d2[0.354] g[17.196]
>16054, d1[0.353] d2[0.349] g[19.481]
>16055, d1[0.356] d2[0.345] g[58.374]
>16056, d1[0.348] d2[0.348] g[16.601]
>16057, d1[0.364] d2[0.355] g[22.533]
>16058, d1[0.351] d2[0.342] g[35.087]
>16059, d1[0.349] d2[0.357] g[14.011]
>16060, d1[0.350] d2[0.342] g[23.072]
>16061, d1[0.350] d2[0.347] g[16.646]
>16062, d1[0.348] d2[0.344] g[18.648]
>16063, d1[0.349] d2[0.326] g[51.158]
>16064, d1[0.343] d2[0.332] g[34.389]
>16065, d1[0.349] d2[0.364] g[71.114]
>16066, d1[0.338] d2[0.341] g[25.432]
>16067, d1[0

>16257, d1[0.341] d2[0.353] g[26.608]
>16258, d1[0.321] d2[0.357] g[16.969]
>16259, d1[0.327] d2[0.346] g[26.688]
>16260, d1[0.350] d2[0.355] g[24.133]
>16261, d1[0.324] d2[0.358] g[34.348]
>16262, d1[0.341] d2[0.361] g[29.876]
>16263, d1[0.345] d2[0.369] g[21.891]
>16264, d1[0.325] d2[0.371] g[24.920]
>16265, d1[0.311] d2[0.415] g[47.254]
>16266, d1[0.342] d2[0.356] g[40.148]
>16267, d1[0.337] d2[0.359] g[15.771]
>16268, d1[0.336] d2[0.354] g[41.667]
>16269, d1[0.328] d2[0.369] g[27.815]
>16270, d1[0.311] d2[0.336] g[17.465]
>16271, d1[0.326] d2[0.369] g[14.653]
>16272, d1[0.338] d2[0.347] g[17.183]
>16273, d1[0.332] d2[0.356] g[23.936]
>16274, d1[0.345] d2[0.369] g[30.082]
>16275, d1[0.344] d2[0.366] g[18.762]
>16276, d1[0.345] d2[0.351] g[15.645]
>16277, d1[0.344] d2[0.358] g[35.930]
>16278, d1[0.341] d2[0.366] g[28.891]
>16279, d1[0.356] d2[0.369] g[26.982]
>16280, d1[0.338] d2[0.352] g[31.416]
>16281, d1[0.348] d2[0.380] g[25.976]
>16282, d1[0.337] d2[0.360] g[16.864]
>16283, d1[0

>16473, d1[0.338] d2[0.332] g[28.221]
>16474, d1[0.342] d2[0.347] g[11.613]
>16475, d1[0.355] d2[0.348] g[41.145]
>16476, d1[0.362] d2[0.341] g[29.288]
>16477, d1[0.335] d2[0.326] g[35.275]
>16478, d1[0.363] d2[0.368] g[40.541]
>16479, d1[0.349] d2[0.362] g[30.341]
>16480, d1[0.334] d2[0.374] g[22.645]
>16481, d1[0.346] d2[0.350] g[29.381]
>16482, d1[0.323] d2[0.366] g[40.127]
>16483, d1[0.344] d2[0.367] g[22.841]
>16484, d1[0.350] d2[0.364] g[24.297]
>16485, d1[0.345] d2[0.349] g[30.183]
>16486, d1[0.348] d2[0.354] g[18.763]
>16487, d1[0.345] d2[0.359] g[45.080]
>16488, d1[0.354] d2[0.352] g[31.230]
>16489, d1[0.335] d2[0.344] g[28.490]
>16490, d1[0.326] d2[0.359] g[31.197]
>16491, d1[0.334] d2[0.349] g[50.001]
>16492, d1[0.323] d2[0.345] g[17.045]
>16493, d1[0.332] d2[0.355] g[40.030]
>16494, d1[0.358] d2[0.347] g[12.552]
>16495, d1[0.341] d2[0.344] g[23.645]
>16496, d1[0.350] d2[0.359] g[27.528]
>16497, d1[0.325] d2[0.352] g[24.521]
>16498, d1[0.346] d2[0.352] g[21.825]
>16499, d1[0

>16689, d1[0.339] d2[0.344] g[32.681]
>16690, d1[0.369] d2[0.358] g[28.073]
>16691, d1[0.317] d2[0.351] g[28.248]
>16692, d1[0.331] d2[0.379] g[61.551]
>16693, d1[0.322] d2[0.359] g[37.382]
>16694, d1[0.337] d2[0.362] g[13.267]
>16695, d1[0.335] d2[0.347] g[34.767]
>16696, d1[0.343] d2[0.357] g[24.635]
>16697, d1[0.361] d2[0.366] g[18.413]
>16698, d1[0.342] d2[0.340] g[29.192]
>16699, d1[0.340] d2[0.355] g[22.240]
>16700, d1[0.327] d2[0.355] g[32.371]
>16701, d1[0.365] d2[0.364] g[25.156]
>16702, d1[0.337] d2[0.347] g[27.157]
>16703, d1[0.331] d2[0.362] g[22.116]
>16704, d1[0.360] d2[0.351] g[24.976]
>16705, d1[0.331] d2[0.352] g[21.603]
>16706, d1[0.345] d2[0.339] g[22.515]
>16707, d1[0.335] d2[0.340] g[32.451]
>16708, d1[0.334] d2[0.351] g[24.626]
>16709, d1[0.349] d2[0.365] g[35.543]
>16710, d1[0.361] d2[0.373] g[43.692]
>16711, d1[0.331] d2[0.360] g[23.482]
>16712, d1[0.340] d2[0.356] g[31.620]
>16713, d1[0.325] d2[0.395] g[35.736]
>16714, d1[0.346] d2[0.341] g[27.979]
>16715, d1[0